In [ ]:
!git clone https://github.com/leonardozilli/FrameTrigger.git
%cd FrameTrigger/

In [1]:
!pip install transformers[torch] datasets -q

In [1]:
from src.fn17 import load_dataset_hf, load_dataset_nltk
from src.train import train, test
from src.predict import predict_triggers
from src.evaluate import compute_metrics
from src.process_data import prepare_data

2024-02-08 22:44:17.866863: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-08 22:44:18.073363: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-08 22:44:19.232892: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
CHECKPOINT = 'bert-base-cased'
TASK = 'frames'

In [3]:
dataset = load_dataset_nltk()
tokenized_dataset = prepare_data(dataset, CHECKPOINT, task=TASK)
tokenized_dataset

[nltk_data] Downloading package framenet_v17 to /home/leo/nltk_data...
[nltk_data]   Package framenet_v17 is already up-to-date!
100%|██████████| 107/107 [00:23<00:00,  4.53it/s]


Map:   0%|          | 0/3425 [00:00<?, ? examples/s]

Map:   0%|          | 0/328 [00:00<?, ? examples/s]

Map:   0%|          | 0/1354 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3425
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 328
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1354
    })
})

## Train

In [4]:
OUT_DIR = './models'
N_EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 2e-5

In [ ]:
train(pretrained_model=CHECKPOINT, task=TASK, dataset=tokenized_dataset,
    epochs=N_EPOCHS, batch_size=BATCH_SIZE, lr=LEARNING_RATE, model_output_path=OUT_DIR)

## Test

In [ ]:
test(OUT_DIR, tokenized_dataset['test'])

## Predict

In [74]:
sentence = "When the moon hits your eye, that's 'amore'."

predict_triggers(OUT_DIR, sentence)

"When* the moon hits* your eye* , that ' s ' amore ' ."

In [ ]:
from spacy import displacy

dic_ents = {
    "text": "My name is John Smith and I live in Paris",
    "ents": [
        {"start": 11, "end": 21, "label": "Person"},
        {"start": 36, "end": 41, "label": "Location"},
    ],
    "title": None
}

displacy.render(dic_ents, manual=True, style="ent")


In [38]:
# dataset
#  - iob
# displacy
dataset['train'][269]

{'id': 1430,
 'sent_id': 1542,
 'tokens': ['Does',
  'Iran',
  'have',
  'the',
  'infrastructure',
  'necessary',
  'to',
  'produce',
  'nuclear',
  'weapons',
  '?'],
 'frame_tags': [0, 0, 112, 0, 134, 192, 0, 9, 662, 662, 0],
 'coarse_pos': ['VVZ',
  'NP',
  'VH',
  'dt',
  'nn',
  'jj',
  'to',
  'VV',
  'jj',
  'nns',
  'sent'],
 'fine_pos': ['NNP',
  'NNP',
  'VB',
  'DT',
  'NN',
  'JJ',
  'TO',
  'VB',
  'JJ',
  'NNS',
  '.'],
 'lemma': ['Does',
  'Iran',
  'have',
  'the',
  'infrastructure',
  'necessary',
  'to',
  'produce',
  'nuclear',
  'weapon',
  '?'],
 'LUs': ['_', '_', 'have.v', '_', '_', '_', '_', '_', '_', '_', '_'],
 'FEs': ['O\n',
  'S-Owner\n',
  'O\n',
  'B-Possession\n',
  'I-Possession\n',
  'I-Possession\n',
  'I-Possession\n',
  'I-Possession\n',
  'I-Possession\n',
  'I-Possession\n',
  'O\n']}